In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import torch.nn as nn
import torch
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/recommendation-system-in-python/movies.csv
/kaggle/input/recommendation-system-in-python/ratings.csv


In [2]:
movies=pd.read_csv('/kaggle/input/recommendation-system-in-python/movies.csv')
ratings=pd.read_csv('/kaggle/input/recommendation-system-in-python/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
genres=[]
for genre in movies['genres']:
    genre=genre.split('|')
    genres.extend([i for i in genre if i not in genres])
print(genres)
len(genres)

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'War', 'Musical', 'Documentary', 'IMAX', 'Western', 'Film-Noir', '(no genres listed)']


20

In [5]:
df=movies.merge(ratings,on='movieId')
df

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483
...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,1537109082
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,1537109545
100833,193585,Flint (2017),Drama,184,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021


In [6]:
def fill_genre_rating(row):
    
    movie_types=row['genres'].split('|')
    genre_list=[np.nan]*len(genres)
    for i in movie_types:
        genre_list[genres.index(i)]=1
    genre_list=[i*row['rating'] for i in genre_list]
    genre_list=pd.Series(genre_list)
    
    return genre_list
df[genres]=df.apply(fill_genre_rating,axis=1)
df

,movieId,title,genres,userId,rating,timestamp,Adventure,Animation,Children,Comedy,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,4.5,4.5,4.5,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,2.5,2.5,2.5,2.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,4.5,4.5,4.5,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,1537109082,NaN,4.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,1537109545,NaN,3.5,NaN,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100833,193585,Flint (2017),Drama,184,3.5,1537109805,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021,NaN,3.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.select_dtypes(['float','int']).groupby('userId',dropna=False).mean().drop(['movieId','rating','timestamp'],axis=1)

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
userId,,,,,,,,,,,,,,,,,,,,
1,4.388235,4.689655,4.547619,4.277108,4.297872,4.307692,4.529412,4.322222,4.355556,4.145455,3.470588,4.166667,4.225000,4.500000,4.681818,NaN,NaN,4.285714,5.0000,NaN
2,4.166667,NaN,NaN,4.000000,NaN,4.500000,3.882353,3.954545,3.800000,3.700000,3.000000,4.000000,3.875000,4.500000,NaN,4.333333,3.750000,3.500000,NaN,NaN
3,2.727273,0.500000,0.500000,1.000000,3.375000,0.500000,0.750000,3.571429,0.500000,4.142857,4.687500,5.000000,4.200000,0.500000,0.500000,NaN,NaN,NaN,NaN,NaN
4,3.655172,4.000000,3.800000,3.509615,3.684211,3.379310,3.483333,3.320000,3.814815,3.552632,4.250000,3.478261,2.833333,3.571429,4.000000,4.000000,3.000000,3.800000,4.0000,NaN
5,3.250000,4.333333,4.111111,3.466667,4.142857,3.090909,3.800000,3.111111,3.833333,3.555556,3.000000,4.000000,2.500000,3.333333,4.400000,NaN,3.666667,3.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,3.503401,3.714286,3.448980,3.565321,3.597938,3.740845,3.787966,3.178808,3.654135,3.525126,3.346154,3.791209,3.556962,3.792308,3.727273,3.800000,3.062500,3.411765,3.8125,NaN
607,3.466667,3.333333,3.421053,3.327273,3.571429,3.517241,4.012195,3.722222,3.814815,4.114754,4.114286,4.647059,3.250000,4.166667,3.600000,NaN,5.000000,4.000000,NaN,NaN
608,3.220994,3.118182,2.460227,2.736620,3.000000,2.886792,3.437500,3.330325,3.613014,3.536680,3.319588,3.550725,3.296407,3.578947,2.757576,3.000000,4.000000,2.636364,3.7500,NaN


In [8]:
df.drop(['title','genres','rating','timestamp','(no genres listed)'],axis=1)

,movieId,userId,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir
0,1,1,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,7,4.5,4.5,4.5,4.5,4.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,15,2.5,2.5,2.5,2.5,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,17,4.5,4.5,4.5,4.5,4.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,193581,184,NaN,4.0,NaN,4.0,4.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100832,193583,184,NaN,3.5,NaN,3.5,3.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100833,193585,184,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100834,193587,184,NaN,3.5,NaN,NaN,NaN,NaN,NaN,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
user_movie=df.pivot(columns='movieId',index='userId',values='rating')
user_movie

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
user_movie.transpose()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
if 'userId' not in user_movie.columns:
    print("userId not found in columns. Resetting index...")
    user_movie.reset_index(inplace=True)  # Convert index to a column

userId not found in columns. Resetting index...


In [12]:
df_long = user_movie.melt(id_vars=['userId'], var_name='movieId', value_name='rating')
df_long

,userId,movieId,rating
0,1,1,4.0
1,2,1,NaN
2,3,1,NaN
3,4,1,NaN
4,5,1,4.0
...,...,...,...
5931635,606,193609,NaN
5931636,607,193609,NaN
5931637,608,193609,NaN
5931638,609,193609,NaN


In [13]:
df_long=df_long.dropna()
df_long

,userId,movieId,rating
0,1,1,4.0
4,5,1,4.0
6,7,1,4.5
14,15,1,2.5
16,17,1,4.5
...,...,...,...
5928773,184,193581,4.0
5929383,184,193583,3.5
5929993,184,193585,3.5
5930603,184,193587,3.5


In [14]:
df_long['userId'] = df_long['userId'].astype(int)
df_long['movieId'] = df_long['movieId'].astype(int)
df_long['rating'] = df_long['rating'].astype('float32')

In [15]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df_long, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
train_user = torch.tensor(train_data["userId"].values, dtype=torch.long)
train_item = torch.tensor(train_data["movieId"].values, dtype=torch.long)
train_rating = torch.tensor(train_data["rating"].values, dtype=torch.float32)

test_user = torch.tensor(test_data["userId"].values, dtype=torch.long)
test_item = torch.tensor(test_data["movieId"].values, dtype=torch.long)
test_rating = torch.tensor(test_data["rating"].values, dtype=torch.float32)

print(f"Train size: {len(train_user)}, Test size: {len(test_user)}")

Train size: 80668, Test size: 20168


In [16]:
from scipy.sparse import coo_matrix

# Get number of users and items
num_users = df_long['userId'].max() + 1  # Assuming user IDs start from 0
num_items = df_long['movieId'].max() + 1  # Assuming movie IDs start from 0

# Create a sparse matrix in COO format (row=user, col=movie, data=rating)
sparse_matrix = coo_matrix(
    (df_long['rating'], (df_long['userId'], df_long['movieId'])),
    shape=(num_users, num_items)
)

In [17]:

class NCF(nn.Module):
    def __init__(self, num_users, num_items, embed_dim=8, mlp_layers=[16, 8, 4]):
        super(NCF, self).__init__()

        # User and Item Embeddings
        self.user_embedding_gmf = nn.Embedding(num_users, embed_dim)
        self.item_embedding_gmf = nn.Embedding(num_items, embed_dim)

        self.user_embedding_mlp = nn.Embedding(num_users, embed_dim)
        self.item_embedding_mlp = nn.Embedding(num_items, embed_dim)

        # MLP Layers
        mlp_input_dim = embed_dim * 2
        mlp_layers_list = []
        for layer in mlp_layers:
            mlp_layers_list.append(nn.Linear(mlp_input_dim, layer))
            mlp_layers_list.append(nn.ReLU())
            mlp_input_dim = layer
        self.mlp_layers = nn.Sequential(*mlp_layers_list)

        # Final Prediction Layer
        self.final_layer = nn.Linear(embed_dim + mlp_layers[-1], 1)

    def forward(self, user, item):
        # GMF branch (Element-wise multiplication)
        gmf_user = self.user_embedding_gmf(user)
        gmf_item = self.item_embedding_gmf(item)
        gmf_output = gmf_user * gmf_item

        # MLP branch (Concatenation and transformation)
        mlp_user = self.user_embedding_mlp(user)
        mlp_item = self.item_embedding_mlp(item)
        mlp_input = torch.cat((mlp_user, mlp_item), dim=1)
        mlp_output = self.mlp_layers(mlp_input)

        # Concatenate GMF and MLP outputs
        final_input = torch.cat((gmf_output, mlp_output), dim=1)
        output = self.final_layer(final_input)
        output = torch.clamp(output, min=1, max=5)
        return output.squeeze()


In [18]:
# Define Model
# config = {
#     'num_users': num_users,
#     'num_items': num_items,
#     'latent_dim_mf': 16,
#     'latent_dim_mlp': 32,
#     '
# layers': [64, 32, 16],


In [19]:
from torch.utils.data import Dataset, DataLoader

class SparseRatingDataset(Dataset):
    def __init__(self, user_ids, item_ids, ratings):
        self.user_ids = user_ids
        self.item_ids = item_ids
        self.ratings = ratings

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.ratings[idx]

# Create dataset and dataloader
dataset = SparseRatingDataset(train_user, train_item, train_rating)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)
for batch in train_loader:
    print(batch)
    break

[tensor([294, 563, 414, 414]), tensor([2796, 2572, 4033,  325]), tensor([3.0000, 3.5000, 4.0000, 2.0000])]


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
model = NCF(num_users, num_items).to(device)  # Move model to GPU

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for user, item, rating in train_loader:
        user = user.to(device)
        item = item.to(device)
        rating = rating.to(device)
        
        optimizer.zero_grad()
        prediction = model(user, item)
        loss = criterion(prediction, rating)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

True
Epoch 1/25, Loss: 22838.5929
Epoch 2/25, Loss: 17048.6310
Epoch 3/25, Loss: 15838.7859
Epoch 4/25, Loss: 15058.4697
Epoch 5/25, Loss: 14378.8805
Epoch 6/25, Loss: 13752.9656
Epoch 7/25, Loss: 13198.7465
Epoch 8/25, Loss: 12608.5733
Epoch 9/25, Loss: 12082.7629
Epoch 10/25, Loss: 11573.5060
Epoch 11/25, Loss: 11081.7430
Epoch 12/25, Loss: 10651.2887
Epoch 13/25, Loss: 10242.1761
Epoch 14/25, Loss: 9863.7445
Epoch 15/25, Loss: 9521.6766
Epoch 16/25, Loss: 9203.9940
Epoch 17/25, Loss: 8902.1949
Epoch 18/25, Loss: 8649.5538
Epoch 19/25, Loss: 8401.1701
Epoch 20/25, Loss: 8183.6155
Epoch 21/25, Loss: 7956.8582
Epoch 22/25, Loss: 7761.1658
Epoch 23/25, Loss: 7584.1968
Epoch 24/25, Loss: 7420.3242
Epoch 25/25, Loss: 7255.4426


In [21]:
import math

# Evaluate Model on Test Set
model.eval()
total_loss = 0
count = 0
test_loader=SparseRatingDataset(test_user,test_item,test_rating)
test_loader = DataLoader(test_loader, batch_size=4, shuffle=True)
with torch.no_grad():
    for user, item, rating in test_loader:
        # Move tensors to the same device as the model (GPU if available)
        user = user.to(device)
        item = item.to(device)
        rating = rating.to(device)

        prediction = model(user, item)

        # Compute loss (ensure dtype matches)
        loss = criterion(prediction, rating)
        total_loss += loss.item()
        count += len(user)

# Compute RMSE
rmse = math.sqrt(total_loss / count)
print(f"Test RMSE: {rmse:.4f}")

Test RMSE: 0.5100
